# Classification models with PySpark

In this notebook we will learn more about classification models using Pyspark library. The main objective is to create a classification models that identifies which clients wants to cancel the service or not.

## Preparing Data

In this section we will be preparing data to feed our model, also an analysis will be done.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark= SparkSession.builder.master('local[*]').getOrCreate()

spark

In [2]:
import zipfile

with zipfile.ZipFile('base de dados.zip', 'r') as zip:
    zip.extractall()

dados = spark.read.csv("base de dados\dados_clientes.csv", sep=',', header=True, inferSchema=True)

In [3]:
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

In [4]:
dados.limit(20).toPandas()

,id,Churn,Mais65anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,MaisDeUmaLinhaTelefonica,Internet,SegurancaOnline,BackupOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,TipoContrato,ContaCorreio,MetodoPagamento,MesesCobrados
0,0,Nao,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,29.85
1,1,Nao,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,Boleto,56.95
2,2,Sim,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,Boleto,53.85
3,3,Nao,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30
4,4,Sim,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,70.70
5,5,Sim,0,Nao,Nao,8,Sim,Sim,FibraOptica,Nao,Nao,Sim,Nao,Sim,Sim,Mensalmente,Sim,BoletoEletronico,99.65
6,6,Nao,0,Nao,Sim,22,Sim,Sim,FibraOptica,Nao,Sim,Nao,Nao,Sim,Nao,Mensalmente,Sim,CartaoCredito,89.10
7,7,Nao,0,Nao,Nao,10,Nao,SemServicoTelefonico,DSL,Sim,Nao,Nao,Nao,Nao,Nao,Mensalmente,Nao,Boleto,29.75
8,8,Sim,0,Sim,Nao,28,Sim,Sim,FibraOptica,Nao,Nao,Sim,Sim,Sim,Sim,Mensalmente,Sim,BoletoEletronico,104.80
9,9,Nao,0,Nao,Sim,62,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,UmAno,Nao,DebitoEmConta,56.15


In [5]:
dados.count()

10348

In [6]:
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [7]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [8]:
dados.groupBy('Internet').count().show()

+-----------+-----+
|   Internet|count|
+-----------+-----+
|FibraOptica| 5401|
|        Nao| 1741|
|        DSL| 3206|
+-----------+-----+



# Data Treatment

In this section we will treat data, change some of the types and change words for binary or integers to represent categories.

In [24]:
colunas_binarias=[
    'Churn',
    'Conjuge',
    'Dependentes',
    'BackupOnline',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [25]:
from pyspark.sql import functions as f

In [26]:
todas_colunas= [f.when(f.col(c) == 'Sim', 1).otherwise(0).alias(c) for c in colunas_binarias]

In [27]:
for coluna in reversed(dados.columns):
    if coluna not in colunas_binarias:
        todas_colunas.insert(0, coluna)
todas_colunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS `Churn`'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS `Conjuge`'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS `Dependentes`'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS `BackupOnline`'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS `TelefoneFixo`'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS `MaisDeUmaLinhaTelefonica`'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS `SegurancaOnline`'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS `SeguroDispositivo`'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS `SuporteTecnico`'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS `TVaCabo`'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS `StreamingFilmes`'>,
 Column

In [28]:
dados.select(
    todas_colunas
).toPandas()

,id,Mais65anos,MesesDeContrato,Internet,TipoContrato,MetodoPagamento,MesesCobrados,Churn,Conjuge,Dependentes,BackupOnline,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,ContaCorreio
0,0,0,1,DSL,Mensalmente,BoletoEletronico,29.850000,0,1,0,1,0,0,0,0,0,0,0,1
1,1,0,34,DSL,UmAno,Boleto,56.950000,0,0,0,0,1,0,1,1,0,0,0,0
2,2,0,2,DSL,Mensalmente,Boleto,53.850000,1,0,0,1,1,0,1,0,0,0,0,1
3,3,0,45,DSL,UmAno,DebitoEmConta,42.300000,0,0,0,0,0,0,1,1,1,0,0,0
4,4,0,2,FibraOptica,Mensalmente,BoletoEletronico,70.700000,1,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,10343,0,4,FibraOptica,Mensalmente,BoletoEletronico,86.687604,1,1,0,0,1,1,0,0,0,1,0,1
10344,10344,1,13,FibraOptica,Mensalmente,BoletoEletronico,86.195233,1,0,0,0,1,1,0,0,0,1,0,1
10345,10345,0,15,FibraOptica,Mensalmente,Boleto,75.099071,1,0,0,0,1,1,0,0,0,0,0,1
10346,10346,0,17,FibraOptica,Mensalmente,CartaoCredito,87.824082,1,0,0,0,1,1,0,0,0,0,1,1


In [29]:
dataset= dados.select(todas_colunas)

In [30]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



# Dummy Variables

In this section we will create dummy variables for those columns with a String type.

In [31]:
dados.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [32]:
dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [33]:
Internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato= dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento= dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [35]:
dataset\
    .join(Internet, 'id', how= 'inner')\
    .join(TipoContrato, 'id', how= 'inner')\
    .join(MetodoPagamento, 'id', how= 'inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    )\
    .drop(
        'Internet', 'TipoContrato', 'MetodoPagamento',
        'DSL', 'FibraOptica', 'Nao',
        'Mensalmente', 'UmAno', 'DoisAnos',
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )\
    .limit(5).toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,Churn,Conjuge,Dependentes,BackupOnline,TelefoneFixo,MaisDeUmaLinhaTelefonica,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,0,0,1,29.85,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
1,1,0,34,56.95,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,2,0,2,53.85,1,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,3,0,45,42.30,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
4,4,0,2,70.70,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,1,0


In [36]:
dataset = dataset\
    .join(Internet, 'id', how= 'inner')\
    .join(TipoContrato, 'id', how= 'inner')\
    .join(MetodoPagamento, 'id', how= 'inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    )\
    .drop(
        'Internet', 'TipoContrato', 'MetodoPagamento',
        'DSL', 'FibraOptica', 'Nao',
        'Mensalmente', 'UmAno', 'DoisAnos',
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

In [37]:
dataset.show()

+---+----------+---------------+-------------+-----+-------+-----------+------------+------------+------------------------+---------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
| id|Mais65anos|MesesDeContrato|MesesCobrados|Churn|Conjuge|Dependentes|BackupOnline|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+---+----------+---------------+-------------+-----+-------+-----------+------------+------------+------------------------